# Problem Statement

We would like to add a feature for recommended tickers and weights. 


## High Level Design

* This can be run weekly/fortnightly. 

* We will download data for the top 50 tickers for a given index. 

* Get the sharpe ratios based on criteria - 5 years data, 2 years data, 1 year data. Average them out and take the top 20, discard the rest

* Use the top 20, run Efficient Frontier for min volatility and max sharpe. Display these data with some text explanation (**OpenAI??**)


In [1]:
import pandas as pd
import json 

from UtilFuncs.get_yfinance_data import *
from UtilFuncs.calculations import *
from UtilFuncs.visualizations_1 import *

In [2]:
index_to_run = "^AXJO"

### Step1: Download all data  

In [ ]:

# Get all tickers for this index
with open('Resources/symbols_list.json', 'r') as json_file:
    symbols_dict = json.load(json_file)

tickers_names = symbols_dict[index_to_run]
# The symbols are retrieved in this format: "NAB.AX: National Australia Bank". 
# We will need to split at the : and take the first part as our ticker codes
tickers_all = [ticker.split(":")[0] for ticker in tickers_names]

**Call yfinance download and fetch prices for 10 years**

In [ ]:
df_prices_all = pd.DataFrame()
# We'll run in batches of 5 to support multithreading
for i in range(10):
    start_index = 5 * i 
    end_index = start_index + 5
    tickers = tickers_all[start_index:end_index]
    print(tickers)

    if len(df_prices_all) == 0: 

        df_prices_all = get_securities_prices(tickers=tickers, download_tnx=False)

    else: 

        df_prices_all = pd.concat([df_prices_all, get_securities_prices(tickers=tickers, download_tnx=False)], axis="columns") 

In [ ]:
print(df_prices_all.shape)

*there are 2529 rows, which is about 10 years X 252 business days per year, so it looks right to me*

**We will also need the index prices for the same period, and append it to the end of the DF**

In [ ]:
df_prices_all = pd.concat([df_prices_all, get_securities_prices(tickers=[index_to_run], download_tnx=False)], axis="columns") 
df_prices_all.head()

In [ ]:
df_prices_all.index.dtype

In [ ]:
df_prices_all.to_csv(f"Resources/Recommendations_{index_to_run}_stocks_data.csv", index=True)

#### Step1 Complete. The price data for 10 years has been saved to the csv file

### Step2: Get the sharpe ratios based on criteria - 5 years data, 2 years data, 1 year data. Average them out and take the top 20, discard the rest

In [3]:
df_prices_all = pd.read_csv(f"Resources/Recommendations_{index_to_run}_stocks_data.csv", index_col=0, parse_dates=True, infer_datetime_format=True)
df_prices_all.head(2)

,ANZ.AX,BHP.AX,CBA.AX,CSL.AX,WBC.AX,FMG.AX,GMG.AX,MQG.AX,NAB.AX,WOW.AX,...,MGR.AX,QAN.AX,TAH.AX,VCX.AX,BLD.AX,EVN.AX,GPT.AX,MPL.AX,SGP.AX,^AXJO
Date,,,,,,,,,,,,,,,,,,,,,
2013-09-09,17.343498,18.547699,45.282280,58.551270,18.952793,1.895105,3.627663,29.029781,17.661514,25.362722,...,1.069150,1.258783,1.911937,1.273060,3.328157,0.658314,2.196738,NaN,2.072585,5181.500000
2013-09-10,17.476728,18.683008,45.448334,58.031803,19.237709,1.920656,3.627663,29.732576,17.799913,25.392027,...,1.062748,1.263377,1.936139,1.256161,3.406551,0.643603,2.196738,NaN,2.094344,5201.200195


In [4]:
# Call the function defined in UtilFuncs.calculations.py
dict_dfs = calculate_metrics(df_prices_all)
dict_dfs.keys()

dict_keys(['combined_returns', 'cum_returns', 'cum_returns_sma', 'cum_returns_std', 'volatility', 'returns_variance', 'cov_matrix', 'df_metrics'])

In [5]:
df_prices_all.shape

(2529, 51)

**Final code**

Call the function defined in UtilFuncs.calculations.py 

In [6]:
df_sharpe_ratios = pd.DataFrame()
col_names = []
for num in [5, 2, 1]: 

    dict = calculate_metrics(df_prices_all, years=num)
    df_sharpe_ratios = pd.concat([df_sharpe_ratios, dict['df_metrics']['Sharpe_Ratio']], axis="columns")
    col_names += [f'Sharpe_Ratio_{num}']
    
df_sharpe_ratios.columns = col_names

Running calculate_metrics on period: 2018-09-06 to 2023-09-05
Running calculate_metrics on period: 2021-09-05 to 2023-09-05
Running calculate_metrics on period: 2022-09-05 to 2023-09-05


In [7]:
df_sharpe_ratios['Sharpe_Ratio_Score'] = df_sharpe_ratios.mean(axis=1)

In [8]:
df_sharpe_scores = df_sharpe_ratios.sort_values(by='Sharpe_Ratio_Score', ascending=False)

##### **Outcome**: We have the dataframe which has the securities sorted by Sharpe Ratios

### Step3: Use the top 20, run Efficient Frontier for min volatility and max sharpe. Display these data

In [15]:
top_20 = df_sharpe_scores.index[:20].to_list() 
print(top_20)

['FMG.AX', 'WDS.AX', 'BXB.AX', 'ORG.AX', 'QBE.AX', 'RIO.AX', 'NCM.AX', 'BHP.AX', 'SUN.AX', 'COH.AX', 'REA.AX', 'CPU.AX', 'EVN.AX', 'GMG.AX', 'BLD.AX', 'WES.AX', 'IAG.AX', 'SGP.AX', 'BSL.AX', 'CBA.AX']


In [16]:
# Slice the prices dataframe to discard rest of the ticker symbols
df_prices_top_20 = df_prices_all[top_20]
df_prices_top_20.head(2)

,FMG.AX,WDS.AX,BXB.AX,ORG.AX,QBE.AX,RIO.AX,NCM.AX,BHP.AX,SUN.AX,COH.AX,REA.AX,CPU.AX,EVN.AX,GMG.AX,BLD.AX,WES.AX,IAG.AX,SGP.AX,BSL.AX,CBA.AX
Date,,,,,,,,,,,,,,,,,,,,
2013-09-09,1.895105,22.043320,6.087206,8.839066,11.589579,39.172737,11.709445,18.547699,8.37161,47.705681,33.855453,7.430946,0.658314,3.627663,3.328157,17.813351,3.673750,2.072585,4.613614,45.282280
2013-09-10,1.920656,21.958973,6.045514,8.934042,11.309604,39.760811,11.399905,18.683008,8.32572,48.247791,34.252201,7.552641,0.643603,3.627663,3.406551,17.843899,3.660653,2.094344,4.667157,45.448334


In [17]:
df_prices_top_20.shape

(2529, 20)

Good, so we now have a dataframe for the historical prices for top 20 securities

Just testing if it works! 

In [18]:
ef_outcomes = run_ef_with_random(df_prices_top_20.copy())

Now to export this dataframe to csv. 

The streamlit app will read from this csv, run the EF simulations and display the outcomes in the **Recommendations**  tab

In [19]:
df_prices_top_20.to_csv("Resources/Recommendations_top_20_prices.csv", index=True)

In [25]:
df_prices = pd.read_csv("Resources/Recommendations_top_20_prices.csv", index_col=0, parse_dates=True, infer_datetime_format=True)
df_prices.iloc[1265:]

,FMG.AX,WDS.AX,BXB.AX,ORG.AX,QBE.AX,RIO.AX,NCM.AX,BHP.AX,SUN.AX,COH.AX,REA.AX,CPU.AX,EVN.AX,GMG.AX,BLD.AX,WES.AX,IAG.AX,SGP.AX,BSL.AX,CBA.AX
Date,,,,,,,,,,,,,,,,,,,,
2018-09-06,1.971626,26.412832,9.481236,6.212648,9.630444,55.762852,16.963047,21.665188,12.183588,189.534164,82.261864,16.247837,2.414775,9.615249,6.311650,29.439671,6.320919,3.079500,15.852583,56.637791
2018-09-07,1.998414,26.087477,9.446632,6.236884,9.604130,56.714706,17.244404,21.596189,12.183588,191.539124,82.649559,16.099968,2.468239,9.615249,6.302582,29.688084,6.329393,3.079500,15.769099,56.936565
2018-09-10,1.928764,26.287125,9.636950,6.317674,9.586590,56.278439,17.380545,21.527191,12.191770,191.641251,80.323517,16.126064,2.441507,9.596950,6.284444,29.728525,6.329393,3.050310,15.908237,57.130363
2018-09-11,1.950195,26.708607,9.834157,6.519644,9.621674,56.571930,17.317015,21.527191,12.085396,192.550934,81.287956,16.578356,2.414775,9.679289,6.230034,29.659203,6.176877,3.064905,15.379511,58.034763
2018-09-12,1.891261,27.137484,9.921807,6.689301,9.586590,56.365692,17.217176,21.458197,11.880837,191.121429,81.694542,16.839296,2.352401,9.725032,6.311650,29.815184,6.193823,3.057608,15.416614,58.083206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-29,19.490147,37.346039,14.150000,8.488608,14.640000,109.879997,25.730000,44.090000,13.120000,266.250000,162.025482,24.740000,3.610000,22.920000,5.030000,52.629997,5.690000,4.210000,20.879999,101.029999
2023-08-30,20.364828,37.630001,15.150000,8.508145,14.700000,112.230003,26.150000,44.730000,13.440000,273.260010,163.099991,25.139999,3.710000,23.200001,4.630000,53.250000,5.770000,4.230000,21.010000,101.839996
2023-08-31,20.374334,37.020000,14.970000,8.508145,14.980000,112.900002,26.030001,44.849998,13.630000,271.489990,165.080002,25.180000,3.710000,23.360001,4.720000,53.869999,5.820000,4.240000,21.010000,102.180000
